<a href="https://colab.research.google.com/github/devcon14/colab-connect/blob/main/backtests/Backtest_NR7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Research, not nr7:

https://thetradingmuse.com/backtesting-stochastic-oscillator-sma-and-wma-trading-strategy-with-bitcoin/



In [1]:
# !pip install quandl
if False:
  # !rm -rf colab-connect
  !git clone https://github.com/devcon14/colab-connect.git
  import sys
  if not "colab-connect" in sys.path:
    sys.path.append("colab-connect")

  # from dataloader import gather_gsheets, gather_av
  from features_kaggle import win_streak_nb, get_features, loss_streak_nb
  from features_candestick import get_features_cnd, get_features_ta
  from dataloader import get_drive_csv, get_av_api

ALPHAVANTAGE_API = "C8RFYYD13O6DPWQ2"
API_GLASSNODE = "b07abeb0-382a-42df-aeed-e72a989f240d"
CC_API_KEY = "4c62b8fbe3270e36eec8a2db57e10460da6b1223b198417f9bc17d1989836751"
QUANDL_API = "Nn-WLPogByiCBZmCJQS6"

In [2]:
import pandas_datareader.data as web
import pandas as pd

symbol = "SPY"

if "/" in symbol:
  sec = web.DataReader(symbol, "av-forex-daily", api_key=ALPHAVANTAGE_API)
else:
  # sec = web.DataReader(symbol, "av-daily-adjusted", api_key=ALPHAVANTAGE_API)
  sec = web.DataReader(symbol, "av-daily-adjusted", api_key=ALPHAVANTAGE_API, start="2010-01-01")
  # sec["close"] = sec["adjusted close"]

sec["symbol"] = symbol
sec

,open,high,low,close,adjusted close,volume,dividend amount,split coefficient,symbol
2010-01-04,112.37,113.3900,111.51,113.33,88.390196,118857369,0.0,1.0,SPY
2010-01-05,113.26,113.6800,112.85,113.63,88.624176,111519230,0.0,1.0,SPY
2010-01-06,113.52,113.9900,113.43,113.71,88.686571,116017127,0.0,1.0,SPY
2010-01-07,113.50,114.3300,113.18,114.19,89.060941,130502017,0.0,1.0,SPY
2010-01-08,113.89,114.6200,113.66,114.57,89.357317,126357911,0.0,1.0,SPY
...,...,...,...,...,...,...,...,...,...
2022-12-19,383.47,383.8200,378.28,380.02,380.020000,79878103,0.0,1.0,SPY
2022-12-20,379.23,382.2300,377.85,380.54,380.540000,74427240,0.0,1.0,SPY
2022-12-21,383.25,387.4100,382.69,386.23,386.230000,78167369,0.0,1.0,SPY
2022-12-22,383.05,386.2134,374.77,380.72,380.720000,100120855,0.0,1.0,SPY


In [3]:
# get_features_cnd(sec)
sec["ft_cnd_range"] = sec["high"] - sec["low"]


In [ ]:
# utility functions
def get_ft_level(price_level, signal_array, index=None):
  level = np.where(signal_array, price_level, None)
  level = pd.Series(level)
  level = level.ffill()
  if not index is None:
    # assigning failed where the dataframe has a date index
    # and the new series has a rangeindex
    # this allows for overriding the index
    level.index = index
  return level

"""
def get_ft_level(price_level, signal_array):
  level = np.where(signal_array, price_level, None)
  return pd.Series(level).ffill()
"""

def bt_convert_frame(sec):
  # prepare dataframe for backtest
  # sec.rename(columns={"open": "Open", "close": "Close", "high": "High", "low": "Low", "volume": "Volume"})
  bt_df = sec.copy()
  bt_df.columns = [col[0].upper() + col[1:] for col in bt_df.columns]
  bt_df.index = pd.to_datetime(bt_df.index)
  return bt_df


In [ ]:
import numpy as np

# using series instead of dataframes
def get_ft_nr_x_bool(crange, period):
  return pd.Series(crange).rolling(period).apply(lambda x: x.iloc[-1] == x.min() if len(x) > 0 else False)

def add_level_pd(name, signal_key, price_key):
  sec[f"chart_{name}_level_{price_key}"] = get_ft_level(sec[price_key], sec[signal_key], index=sec.index)
  sec[f"ft_{name}_level_{price_key}_pd"] = sec["close"] / sec[f"chart_{name}_level_{price_key}"]
  # del sec[f"chart_{name}_level"]

if True:
  sec["ft_ta_NR7"] = get_ft_nr_x_bool(sec["ft_cnd_range"], 7)
  add_level_pd("NR7", "ft_ta_NR7", "high")
  add_level_pd("NR7", "ft_ta_NR7", "low")

  sec["signal_buy_stop"] = sec["chart_NR7_level_high"]
  sec["signal_sell_stop"] = sec["chart_NR7_level_low"]
  
  # [ft_ta_NR7	chart_NR7_level_high	ft_NR7_level_high_pd	chart_NR7_level_low	ft_NR7_level_low_pd	signal_buy_stop	signal_sell_stop]

  display(sec.tail(100))

,open,high,low,close,adjusted close,volume,dividend amount,split coefficient,symbol,ft_cnd_range,ft_ta_NR7,chart_NR7_level_high,ft_NR7_level_high_pd,chart_NR7_level_low,ft_NR7_level_low_pd,signal_buy_stop,signal_sell_stop
2022-08-01,409.15,413.410,408.400,410.77,407.183685,69997471,0.0000,1.0,SPY,5.010,0.0,396.47,1.036068,393.21,1.044658,396.47,393.21
2022-08-02,409.12,413.000,406.820,408.06,404.497346,63435418,0.0000,1.0,SPY,6.180,0.0,396.47,1.029233,393.21,1.037766,396.47,393.21
2022-08-03,410.30,415.680,410.000,414.45,410.831556,67820555,0.0000,1.0,SPY,5.680,0.0,396.47,1.045350,393.21,1.054017,396.47,393.21
2022-08-04,414.37,415.090,412.440,414.17,410.554001,45656574,0.0000,1.0,SPY,2.650,1.0,415.09,0.997784,412.44,1.004195,415.09,412.44
2022-08-05,409.66,414.150,409.600,413.47,409.860113,56814929,0.0000,1.0,SPY,4.550,0.0,415.09,0.996097,412.44,1.002497,415.09,412.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-14,401.61,405.500,396.310,399.40,397.552218,108111349,0.0000,1.0,SPY,9.190,0.0,395.64,1.009504,391.97,1.018956,395.64,391.97
2022-12-15,394.30,395.250,387.885,389.63,387.827418,117705889,0.0000,1.0,SPY,7.365,0.0,395.64,0.984809,391.97,0.994030,395.64,391.97
2022-12-16,385.18,386.575,381.040,383.27,383.270000,119857952,1.7814,1.0,SPY,5.535,0.0,395.64,0.968734,391.97,0.977804,395.64,391.97
2022-12-19,383.47,383.820,378.280,380.02,380.020000,79878103,0.0000,1.0,SPY,5.540,0.0,395.64,0.960520,391.97,0.969513,395.64,391.97


In [ ]:
# primitive backtest on nr7
# TODO with entry and exit levels created we could backtest completely in pandas?

# https://www.youtube.com/watch?v=2xFr0tJU0pM
try:
  import panel as pn
  import hvplot.pandas
  import holoviews as hv
except:
  !pip install hvplot
  import hvplot.pandas
  import holoviews as hv

# pn.extension(comms="colab")
# hv.extension("bokeh")

# FIXME how possible no losers?
# check for gap up
if True:
  #  and x["ft_ta_NR7"]
  trades = sec.apply(lambda x: x["close"] - x["chart_NR7_level_high"] if x["high"] > x["chart_NR7_level_high"] else 0, axis="columns")
  # sum([t for t in trades if t is not None])

  losses = (trades < 0).sum()
  wins = (trades >= 0).sum()
  print (f"Winrate: {wins / trades.count()}")
  print (f"Avg: {trades.mean()}")
  trades

  sec["trades"] = trades
  sec["trades_cumsum"] = trades.cumsum()

# sec[["high", "close", "chart_NR7_level_high", "ft_ta_NR7", "trades"]]
# [x for x in trades if x < 0]
# trades


# trades.cumsum().plot()
sec.hvplot.bar(y="trades_cumsum")


Winrate: 0.8937212863705972
Avg: 0.7575957506891273


:Bars   [index]   (trades_cumsum)

In [ ]:
if False:
  # Toby Crabel and Linda Rashe? opening range breakout
  sec["signal_long"] = (sec.ft_NR7_level_high_pd > 1.0) & (sec.ft_NR7_level_high_pd.shift(1) < 1.0)
  sec["signal_short"] = (sec.ft_NR7_level_low_pd < 1.0) & (sec.ft_NR7_level_low_pd.shift(1) > 1.0)


In [ ]:
bt_df = bt_convert_frame(sec)
bt_df


,Open,High,Low,Close,Adjusted close,Volume,Dividend amount,Split coefficient,Symbol,Ft_cnd_range,Ft_ta_NR7,Chart_NR7_level_high,Ft_NR7_level_high_pd,Chart_NR7_level_low,Ft_NR7_level_low_pd,Signal_buy_stop,Signal_sell_stop,Trades,Trades_cumsum
2010-01-04,112.37,113.390,111.510,113.33,88.390196,118857369,0.0000,1.0,SPY,1.880,NaN,113.39,0.999471,111.51,1.016321,113.39,111.51,0.00,0.000000
2010-01-05,113.26,113.680,112.850,113.63,88.624176,111519230,0.0000,1.0,SPY,0.830,NaN,113.68,0.999560,112.85,1.006912,113.68,112.85,0.00,0.000000
2010-01-06,113.52,113.990,113.430,113.71,88.686571,116017127,0.0000,1.0,SPY,0.560,NaN,113.99,0.997544,113.43,1.002468,113.99,113.43,0.00,0.000000
2010-01-07,113.50,114.330,113.180,114.19,89.060941,130502017,0.0000,1.0,SPY,1.150,NaN,114.33,0.998775,113.18,1.008924,114.33,113.18,0.00,0.000000
2010-01-08,113.89,114.620,113.660,114.57,89.357317,126357911,0.0000,1.0,SPY,0.960,NaN,114.62,0.999564,113.66,1.008006,114.62,113.66,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-14,401.61,405.500,396.310,399.40,397.552218,108111349,0.0000,1.0,SPY,9.190,0.0,395.64,1.009504,391.97,1.018956,395.64,391.97,3.76,2473.550126
2022-12-15,394.30,395.250,387.885,389.63,387.827418,117705889,0.0000,1.0,SPY,7.365,0.0,395.64,0.984809,391.97,0.994030,395.64,391.97,0.00,2473.550126
2022-12-16,385.18,386.575,381.040,383.27,383.270000,119857952,1.7814,1.0,SPY,5.535,0.0,395.64,0.968734,391.97,0.977804,395.64,391.97,0.00,2473.550126
2022-12-19,383.47,383.820,378.280,380.02,380.020000,79878103,0.0000,1.0,SPY,5.540,0.0,395.64,0.960520,391.97,0.969513,395.64,391.97,0.00,2473.550126


In [ ]:
try:
  from backtesting import Backtest, Strategy
  from backtesting.lib import crossover
except Exception as e:
  !pip install backtesting
  from backtesting import Backtest, Strategy
  from backtesting.lib import crossover

class ManualNR(Strategy):
  nr_period = 7

  def init(self):
    self.is_signal = self.I(get_ft_nr_x_bool, self.data.Ft_cnd_range, self.nr_period, overlay=False)
    self.buy_stop = self.I(get_ft_level, self.data.High, self.is_signal.s)
    self.sell_stop = self.I(get_ft_level, self.data.Low, self.is_signal.s)

  def next(self):
    if not self.position:
      if self.is_signal:
        self.buy(stop=self.data["High"])
        self.sell(stop=self.data["Low"])

    else:
      # iterate open trades, close any stop entries on same day
      for trade in self.trades:
        trade.close()

bt = Backtest(bt_df,
              ManualNR,
              cash=10000,
              trade_on_close=True,
              exclusive_orders=True,
              commission=0.001
              )

output = bt.run()
bt.plot()


Row(id='3534', ...)

In [ ]:
bt._results
# dir(bt)

Start                     2010-01-04 00:00:00
End                       2022-12-19 00:00:00
Duration                   4732 days 00:00:00
Exposure Time [%]                    9.558824
Equity Final [$]                 14277.937197
Equity Peak [$]                  14335.752557
Return [%]                          42.779372
Buy & Hold Return [%]              235.321627
Return (Ann.) [%]                    2.787685
Volatility (Ann.) [%]                3.881347
Sharpe Ratio                         0.718226
Sortino Ratio                        1.525927
Calmar Ratio                         0.476424
Max. Drawdown [%]                   -5.851263
Avg. Drawdown [%]                   -1.474193
Max. Drawdown Duration     1402 days 00:00:00
Avg. Drawdown Duration      161 days 00:00:00
# Trades                                  312
Win Rate [%]                        47.435897
Best Trade [%]                       3.882157
Worst Trade [%]                     -2.765388
Avg. Trade [%]                    

In [ ]:
opt = bt.optimize(
    nr_period=range(2,12)
)
opt._strategy

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

<Strategy ManualNR(nr_period=5)>

In [ ]:
"""
# Get help
"""
# bt_df.columns
from backtesting import backtesting as bt
# help(bt.Position)
# bt.Trade